In [11]:
import json
import requests
import pandas as pd
import fake_useragent
import numpy as np
from typing import List
from bs4 import BeautifulSoup

## Загрузка данных

In [12]:
def get_all_regions() -> List[str]:
    user_agent = fake_useragent.UserAgent()
    regions = requests.get(
        url="https://api.hh.ru/areas",
        headers={"user_agent": user_agent.random}
    ).json()
    return [str(region["id"]) for region in regions[0]["areas"]]

In [102]:
def get_count_vacancies(job_title: str, regions: List[str]) -> dict[str, dict[str, int]]:
    mean_salary_by_region = []
    result_row = {}
    for region in get_all_regions():
        url = f"https://api.hh.ru/vacancies?clusters=true&only_with_salary=true&enable_snippets=true&st=searchVacancy' \
            '&text={job_title}&search_field=name&per_page=100&area={region}"
        vacancies = requests.get(
            url=url                    
        ).json()
        for vacancy in vacancies["items"]:
            if vacancy["salary"]["from"] is None:
                continue
            mean_salary_by_region.append(vacancy["salary"]["from"])
    result_row[job_title] = {
        "Средняя зарплата.": int(sum(mean_salary_by_region) / len(mean_salary_by_region)),
        "Количество вакансий в России.": len(mean_salary_by_region)
    }
    return result_row

In [103]:
jt = "Программист"
get_count_vacancies(job_title=jt, regions=get_all_regions())

{'Программист': {'Средняя зарплата.': 90472,
  'Количество вакансий в России.': 3344}}